# Simulation

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad, dblquad
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/MF796

/content/drive/MyDrive/Colab Notebooks/MF796


##Data

In [4]:
cdatas = pd.read_csv('cdatas.csv')
pdatas = pd.read_csv('pdatas.csv')
df = pd.read_excel('spy_call_put.xlsx')

In [5]:
# call price
cp = cdatas[cdatas.columns[2:26:2]]
# call vol
csig = cdatas[cdatas.columns[3::2]]/100
# put price
pp = pdatas[pdatas.columns[2:26:2]]
# put vol
psig = pdatas[pdatas.columns[3::2]]/100
# initial price
s0 = (cdatas[cdatas.columns[0]] + cdatas[cdatas.columns[1]]).values[0]
# maturity
T = np.array([3, 14, 31, 45, 66, 73, 94, 106, 122, 136, 157, 167])
t = T/365
# call strike
ck = cdatas[cdatas.columns[0]].values
# put strike
pk = pdatas[pdatas.columns[0]].values
rs = list()
divs = list()
a = df[df.columns[0]].values[::2]
for i in range(len(a)):
    if 'R' in a[i]:
        k = a[i].find('R')
        r = float(a[i][k + 2: k + 6])
        rs.append(r)

    if 'IDiv' in a[i]:
        k = a[i].find('IDiv')
        div = float(a[i][k + 5:k + 9])
        divs.append(div)
    else:
       divs.append(0)
# expected growth
r = np.array(rs)/100
# dividend rate
q = np.array(divs)/100

##Paths

In [6]:
def simu(s0, k, t, r, q, sig, dt, N):
    M = int(t/dt) # number simu steps # N is the number of simu times

    w = np.random.normal(0, 1, M*N)
    W = w.reshape(N, M)*sig*np.sqrt(dt)

    S = np.zeros([N, M + 1])
    S[:, 0] = s0

    for i in range(M):
        S[:, i + 1] = S[:, i]*((r - q)*dt + W[:, i] + 1)

    return S

##Price

### Barrier

In [7]:
def barr(s0, k, t, r, q, sig, dt, N, bar, knock, opt, St = 1):
    if St == 1:
        S = simu(s0, k, t, r, q, sig, dt, N)
    else:
        S = St

    Sma = S.max(1)
    Smi = S.min(1)
    M = int(t/dt)

    # crude
    if knock == 'in':
        ter = np.minimum((Sma >= bar) + (Smi <= bar), 1)*S[:, -1]
    else:
        ter = np.minimum((Sma < bar) + (Smi > bar), 1)*S[:, -1]
    if opt == 'call':
        p1 = np.mean(np.maximum(ter - k, 0)*np.exp(-(r - q)*t))
    else:
        p1 = np.mean(np.maximum(k - ter, 0)*np.exp(-(r - q)*t))

    # corrected
    s1 = np.log(S[:, :-1])
    s2 = np.log(S[:, 1:])
    p = np.zeros([N, M])
    b = np.log(bar)

    for i in range(M):
        p[:, i] = np.exp(-2*(s1[:, i] - b)*(s2[:, i] - b)/(dt*sig**2))

    if knock == 'in':
        P = 1 - np.prod(np.maximum(1 - p, 0), 1)
    else:
        P = np.prod(np.maximum(1 - p, 0), 1)
    if opt == 'call':
        p2 = np.mean(np.maximum(S[:, -1] - k, 0)*P*np.exp(-(r - q)*t))
    else:
        p2 = np.mean(np.maximum(k - S[:, -1], 0)*P*np.exp(-(r - q)*t))

    return p1, p2

### Testing

In [8]:
def varcomp(s0, k, t, r, q, sig, bar, opt, num, p):
    ipcr1 = []
    ipco1 = []
    ipcr2 = []
    ipco2 = []

    for b in bar:
        bcr1 = []
        bco1 = []
        bcr2 = []
        bco2 = []

        for i in range(len(t)):
            si = sig[sig.columns[i]].values

            cr1, co1 = barr(s0, k[num], t[i], r[i], q[i], si[num], 0.001, 1000, b, 'out', opt)
            bcr1.append(cr1)
            bco1.append(co1)

            cr2, co2 = barr(s0, k[num], t[i], r[i], q[i], si[num], 0.001, 1000, b, 'in', opt)
            bcr2.append(cr2)
            bco2.append(co2)

        ipcr1.append(bcr1)
        ipco1.append(bco1)
        ipcr2.append(bcr2)
        ipco2.append(bco2)

    s1 = np.sqrt(((np.array(ipcr1) + np.array(ipcr2) - p.iloc[num].values)**2).mean(0))
    s2 = np.sqrt(((np.array(ipco1) + np.array(ipco2) - p.iloc[num].values)**2).mean(0))

    return [s1, s1.mean(), s2, s2.mean()]

In [9]:
b1 = [510, 530, 550, 600, 650]
b2 = [490, 470, 450, 400, 350]
num = 12

In [12]:
varcomp(s0, pk, t, r, q, psig, b1, 'put', num, cp)

[array([ 63.18327749, 143.67288383, 179.12009753, 200.50268942,
        217.78117822, 221.60908648, 237.59021971, 246.05861654,
        253.85441149, 262.04908464, 264.2441319 , 265.55210107]),
 212.93481486025826,
 array([0.64097681, 0.31002844, 1.33505873, 2.21406387, 3.15020114,
        3.21628134, 4.52560786, 4.86520204, 7.26576437, 6.90124125,
        9.24902006, 6.49940989]),
 4.181071316575826]

##Hedging

### Smooth Function

In [13]:
def H(x, ep, opt = 'call'):
    if opt == 'put':
        a = -1
    else:
        a = 1
    return a*0.5*(np.tanh(x/ep) + 1)

def R(x, ep):
    return quad(H, -np.inf, x, args = ep)[0]

def dH(x, ep, opt = 'call'):
    if opt == 'put':
        a = -1
    else:
        a = 1
    return a*(1 - np.tanh(x/ep)**2)/(2*ep)

def hH(x, ep):
    return -np.tanh(x/ep)*(1 - np.tanh(x/ep)**2)/(ep**2)

### Data Transform

In [14]:
def S(s0, r, q, sig, t, u):
    return s0*np.exp((r - q - 0.5*sig**2)*t + sig*np.sqrt(t)*norm.ppf(u))

def Smi(s0, s, r, sig, t, u):
    fac = (np.log(s0) - np.log(s))**2 - 2*t*np.log(u)*sig**2
    return np.exp(0.5*(np.log(s0) + np.log(s) - np.sqrt(np.maximum(fac, 0))))

def Sma(s0, s, r, sig, t, u):
    fac = (np.log(s) - np.log(s0))**2 - 2*t*np.log(u)*sig**2
    return np.exp(0.5*(np.log(s0) + np.log(s) + np.sqrt(np.maximum(fac, 0))))

In [15]:
def facd(s0, r, q, sig, t, k, b, u1, u2, ep, Sm, type, opt):
    s = S(s0, r, q, sig, t, u1)

    sm = Sm(s0, s, r, sig, t, u2)

    if type in ['down-out', 'up-in']:
        inpH = sm - b
    else:
        inpH = b - sm

    if opt == 'call':
        inpR = s - k
    else:
        inpR = k - s

    facd1 = sm*dH(inpH, ep, opt)*R(inpR, ep)/s0
    facd2 = s*H(inpH, ep)*H(inpR, ep, opt)/s0

    return facd1 + facd2

### Integration

In [16]:
def intgcor(s0, r, q, si, t, k, bar, ep, Sm, type, opt):
    u1 = (np.array(range(1, 1000)) - 0.5)/1000
    u2 = (np.array(range(1, 1000)) - 0.5)/1000
    U = np.zeros([len(u1), len(u2)])

    for i in range(len(u1)):
        res = facd(s0, r, q, si, t, k, bar, u1[i], u2, ep, Sm, type, opt)
        U[i, :] = res

    delc = np.exp(-t*(r - q))*U.sum()*(0.001)**2

    return delc

### Test Hedging

In [17]:
def bsdelta(s, k, vol, t, r, q, opt):
    d1 = (np.log(s/k) + t*(r - q + 0.5*(vol**2)))/(vol*np.sqrt(t))

    if opt == 'call':
        d = np.exp(-q*t)*norm.cdf(d1)
    else:
        d = -np.exp(-q*t)*norm.cdf(-d1)
    return d

In [18]:
SPY = pd.read_excel('SPY.xlsx')
csen = pd.read_csv('cdatas_sen.csv')
psen = pd.read_csv('pdatas_sen.csv')

In [19]:
rf = np.append(4.0827/100, SPY[SPY.columns[-1]].values/100)[:-1]
pr = np.append(514.865, SPY[SPY.columns[-2]].values)[:-1]
date = np.append(np.datetime64('2024-03-07'), SPY[SPY.columns[0]].values)
ts = ((date[-1] - date[:-1])/(24*60*60*1e9)).astype(float)/365
gp = ((date[1:] -date[:-1])/(24*60*60*1e9)).astype(float)/365
cvol = (csen[csen.columns[2:-1:2]].iloc[18].values/100)[::-1]
pvol = (psen[psen.columns[2:-1:2]].iloc[22].values/100)[::-1]
rr = r[0]

In [37]:
uod = list()
bsd = list()
B = 460
for i in range(len(rf)):
    a = intgcor(pr[0], rr, 0, cvol[i], ts[i], 512, B, 0.02, Smi, 'down-in', 'call')
    b = bsdelta(pr[0], 512, cvol[i], ts[i], rr, 0, 'call')
    uod.append(a)
    bsd.append(b)

In [42]:
ass = 0
mon = 0
P = 0

for i in range(len(uod)):
    P = np.minimum(P, pr[i])

    if P <= B:
        ass = uod[i]
        if i >= 1:
            mon = -(uod[i] - uod[i - 1])*pr[i] + mon*np.exp(rf[i]*gp[i])
        else:
            mon = -uod[i]*pr[i] + mon*np.exp(rf[i]*gp[i])
        continue

    mon = mon*np.exp(rf[i]*gp[i]) + ass*pr[i]
    ass = 0


In [43]:
ass*s0 + mon*np.exp((1/365)*(SPY[SPY.columns[-1]].values)[-1]/100) - np.maximum(s0 - 512, 0)

0.0

In [44]:
ass = 0
mon = 0
P = 0

for i in range(len(bsd)):
    P = np.minimum(P, pr[i])

    if P <= B:
        ass = bsd[i]
        if i >= 1:
            mon = -(bsd[i] - bsd[i - 1])*pr[i] + mon*np.exp(rf[i]*gp[i])
        else:
            mon = -bsd[i]*pr[i] + mon*np.exp(rf[i]*gp[i])
        continue

    mon = mon*np.exp(rf[i]*gp[i]) + ass*pr[i]
    ass = 0

In [45]:
ass*s0 + mon*np.exp((1/365)*(SPY[SPY.columns[-1]].values)[-1]/100) - np.maximum(s0 - 512, 0)

-9.160470841844017